In [ ]:
# import requirements 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

# Load dataset
data = pd.read_csv('Divar.csv')
data.shape


(1000000, 61)

In [31]:
# analyze dataset
data['construction_year'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: construction_year
Non-Null Count   Dtype 
--------------   ----- 
815828 non-null  object
dtypes: object(1)
memory usage: 7.6+ MB


In [16]:
# Identify numeric columns
numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()

# len(numeric_columns)
for i, col in enumerate(numeric_columns, 1):
    print(f"{i:2d}. {col}")


 1. Unnamed: 0
 2. rent_value
 3. price_value
 4. credit_value
 5. transformable_credit
 6. transformed_credit
 7. transformable_rent
 8. transformed_rent
 9. land_size
10. building_size
11. regular_person_capacity
12. cost_per_extra_person
13. rent_price_on_regular_days
14. rent_price_on_special_days
15. rent_price_at_weekends
16. location_latitude
17. location_longitude
18. location_radius


In [20]:
numeric_stats = data[numeric_columns].describe()
numeric_stats

,Unnamed: 0,rent_value,price_value,credit_value,transformable_credit,transformed_credit,transformable_rent,transformed_rent,land_size,building_size,regular_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
count,1000000.000000,3.513220e+05,5.683460e+05,3.520950e+05,3.520850e+05,7.240900e+04,3.512480e+05,7.240900e+04,1.863960e+05,9.803940e+05,29870.000000,1.024100e+04,1.806800e+04,1.046300e+04,1.355100e+04,655608.000000,655608.000000,339699.000000
mean,499999.500000,4.102299e+10,1.736537e+10,4.872084e+10,4.872222e+10,8.557025e+09,4.103164e+10,1.619934e+07,4.165480e+03,4.440648e+03,6.557650,1.209785e+10,1.389016e+11,2.355548e+10,3.156551e+10,34.982108,51.629743,465.149147
std,288675.278933,3.807534e+12,5.878739e+11,4.341346e+12,4.341407e+12,2.064576e+12,3.807935e+12,5.217890e+07,1.218927e+05,1.367118e+05,7.698655,1.103482e+12,7.042335e+12,1.542049e+12,2.434942e+12,2.379169,3.160920,125.896250
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,23.626478,40.162369,0.000000
25%,249999.750000,1.111110e+05,1.400000e+09,1.000000e+08,1.000000e+08,2.000000e+08,1.111110e+05,1.000000e+06,1.100000e+02,7.500000e+01,3.000000,5.000000e+04,4.000000e+05,6.000000e+05,5.500000e+05,34.553551,50.677175,500.000000
50%,499999.500000,5.000000e+06,2.840000e+09,2.500000e+08,2.500000e+08,4.000000e+08,5.000000e+06,6.000000e+06,1.950000e+02,1.030000e+02,4.000000,1.000000e+05,8.000000e+05,1.200000e+06,1.100000e+06,35.723312,51.345791,500.000000
75%,749999.250000,1.200000e+07,5.900000e+09,5.000000e+08,5.000000e+08,8.500000e+08,1.200000e+07,1.500000e+07,2.800000e+02,1.650000e+02,7.000000,2.000000e+05,1.600000e+06,2.500000e+06,2.500000e+06,36.307013,51.805291,500.000000
max,999999.000000,1.000000e+15,1.000000e+14,1.000000e+15,1.000000e+15,5.555556e+14,1.000000e+15,3.000000e+09,1.000000e+07,1.000000e+07,50.000000,1.111111e+14,5.006007e+14,1.111111e+14,2.002503e+14,40.358055,74.511620,500.000000


In [ ]:
# Check missing values in numeric columns
missing_numeric = data[numeric_columns].isnull().sum()
missing_percentage = (missing_numeric / len(data)) * 100
missing_df = pd.DataFrame({
    "column": missing_numeric.index,
    "missing_count": missing_numeric.values,
    "missing_percent": missing_percentage.values
})
# Sort by missing percentage
missing_df = missing_df.sort_values('missing_percent', ascending=False)
print(missing_df.to_string())


                        column  missing_count  missing_percent
11       cost_per_extra_person         989759          98.9759
13  rent_price_on_special_days         989537          98.9537
14      rent_price_at_weekends         986449          98.6449
12  rent_price_on_regular_days         981932          98.1932
10     regular_person_capacity         970130          97.0130
5           transformed_credit         927591          92.7591
7             transformed_rent         927591          92.7591
8                    land_size         813604          81.3604
17             location_radius         660301          66.0301
6           transformable_rent         648752          64.8752
1                   rent_value         648678          64.8678
4         transformable_credit         647915          64.7915
3                 credit_value         647905          64.7905
2                  price_value         431654          43.1654
16          location_longitude         344392          

In [23]:
data

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,rent_value,rent_to_single,rent_type,price_mode,price_value,credit_mode,credit_value,rent_credit_transform,transformable_price,transformable_credit,transformed_credit,transformable_rent,transformed_rent,land_size,building_size,deed_type,has_business_deed,floor,rooms_count,total_floors_count,unit_per_floor,has_balcony,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,has_water,has_warm_water_provider,has_electricity,has_gas,has_heating_system,has_cooling_system,has_restroom,has_security_guard,has_barbecue,building_direction,has_pool,has_jacuzzi,has_sauna,floor_material,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,مشاور املاک,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,NaN,NaN,NaN,سه,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,مشاور املاک,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات,NaN,NaN,NaN,NaN,مقطوع,8.500000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,3,یک,NaN,NaN,NaN,True,True,True,۱۳۸۴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,26000000.0,NaN,NaN,NaN,NaN,مقطوع,7.500000e+08,False,False,7.500000e+08,NaN,26000000.0,NaN,NaN,132.0,NaN,NaN,3,سه,NaN,NaN,NaN,True,True,True,۱۴۰۱,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,NaN,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری,مقطوع,95000000.0,NaN,NaN,NaN,NaN,مقطوع,9.500000e+08,False,False,9.500000e+08,NaN,95000000.0,NaN,NaN,90.0,NaN,NaN,4,یک,NaN,NaN,NaN,True,False,True,۱۴۰۰,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,مشاور املاک,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری,NaN,NaN,NaN,NaN,مقطوع,5.750000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0,single_page,NaN,4,دو,6,NaN,true,True,True,True,۱۴۰۳,NaN,NaN,package,NaN,NaN,shoofaj,air_conditioner,squat_seat,NaN,NaN,north,NaN,NaN,NaN,ceramic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,residential-sell,apartment-sell,kermanshah,NaN,2024-07-01 00:00:00,مشاور املاک,~~~مشاورین املاک قبادی~~~\n■جنوبی تک واحدی\n■د...,آپارتمان ۱۸۰ متری وحدت غربی,NaN,NaN,NaN,NaN,مقطوع,7.470000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,4.0,چهار,NaN,NaN,NaN,True,True,True,۱۴۰۳,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.350235,47.083241,500.0
999996,999996,residential-rent,apartment-rent,tehran,darya,2024-07-01 00:00:00,مشاور املاک,نوساز \n\n تک واحدی\n\nشخصی ساز\n\nروف گا...,آپارتمان ۱۱۰ متری سعادت آباد دریا,مقطوع,45000000.0,NaN,rent_credit,NaN,NaN,مقطوع,1.000000e+09,True,True,1.000000e+09,3.000000e+09,45000000.0,100000.0,NaN,110.0,NaN,NaN,1.0,دو,NaN,NaN,NaN,True,True,True,۱۴۰۳,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [34]:
columns_to_check = ['floor', 'total_floors_count', 'unit_per_floor', 
                   'rooms_count', 'construction_year']

for col in columns_to_check:
    if col in data.columns:
        print(f"\n--- {col} ---")
        print(f"Data type: {data[col].dtype}")
        print(f"Unique values sample: {data[col].unique()[:10]}")
        print(f"Missing values: {data[col].isnull().sum()}")


--- floor ---
Data type: object
Unique values sample: [nan '3' '4' '2' '1' '5' '0' '30' '6' '7']
Missing values: 458252

--- total_floors_count ---
Data type: object
Unique values sample: [nan '6' '3' '14' '5' '4' '2' '7' '8' '10']
Missing values: 695648

--- unit_per_floor ---
Data type: object
Unique values sample: [nan '2' '4' '1' '3' '5' '6' '8' 'more_than_8' '7']
Missing values: 697717

--- rooms_count ---
Data type: object
Unique values sample: ['سه' 'یک' 'دو' 'بدون اتاق' nan 'چهار' 'پنج یا بیشتر']
Missing values: 154101

--- construction_year ---
Data type: object
Unique values sample: [nan '۱۳۸۴' '۱۴۰۱' '۱۴۰۰' '۱۴۰۳' '۱۳۸۹' '۱۳۹۵' '۱۳۹۳' '۱۳۹۶' '۱۳۸۷']
Missing values: 184172


In [ ]:
import re
def clean_persian_numbers(text):
    """translate persian numbers to english numbers"""
    if pd.isna(text):
        return text
    
    persian_to_english = {
        '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
        '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'
    }
    
    text = str(text)
    for persian, english in persian_to_english.items():
        text = text.replace(persian, english)
    
    return text

def convert_persian_words_to_numbers(text):
    """translate persian number words to digits"""
    if pd.isna(text):
        return text
    
    text = str(text).strip()
    
    word_to_number = {
        'یک': '1', 'دو': '2', 'سه': '3', 'چهار': '4',
        'پنج': '5', 'شش': '6', 'هفت': '7', 'هشت': '8',
        'نه': '9', 'ده': '10'
    }
    
    for word, number in word_to_number.items():
        if word in text:
            return number
    
    return text

def extract_year(text):
    """extract year from text"""
    if pd.isna(text):
        return np.nan
    
    text = str(text)
    
    numbers = re.findall(r'\d{4}', text)
    if numbers:
        year = int(numbers[0])
        if 1300 <= year <= 1403:
            return year
    
    return np.nan

# cleaning the floor column
if 'floor' in data.columns:
    print("Cleaning 'floor' column...")
    
    # convert to string
    data['floor'] = data['floor'].astype(str)
    
    # trandslate persian numbers to english
    data['floor'] = data['floor'].apply(clean_persian_numbers)
    
    # keep only numeric characters
    data['floor'] = data['floor'].apply(lambda x: re.sub(r'[^\d.-]', '', str(x)))
    
    # convert to numeric
    data['floor'] = pd.to_numeric(data['floor'], errors='coerce')
    
    print(f"  - Valid values: {data['floor'].notna().sum()}")
    print(f"  - Unique values: {data['floor'].dropna().unique()[:10]}")

# cleaning the rooms_count column
if 'rooms_count' in data.columns:
    print("\nCleaning 'rooms_count' column...")
    
    # convert to string
    data['rooms_count'] = data['rooms_count'].astype(str)
    
    # trandslate persian words to english
    data['rooms_count'] = data['rooms_count'].apply(convert_persian_words_to_numbers)
    
    # trandslate persian numbers to english
    data['rooms_count'] = data['rooms_count'].apply(clean_persian_numbers)

    # keep only numeric characters
    data['rooms_count'] = data['rooms_count'].apply(lambda x: re.sub(r'[^\d.-]', '', str(x)))
    
    # convert to numeric
    data['rooms_count'] = pd.to_numeric(data['rooms_count'], errors='coerce')
    
    print(f"  - Valid values: {data['rooms_count'].notna().sum()}")
    print(f"  - Unique values: {sorted(data['rooms_count'].dropna().unique())[:10]}")

# cleaning the construction_year column
if 'construction_year' in data.columns:
    print("\nCleaning 'construction_year' column...")
    
    data['construction_year'] = data['construction_year'].astype(str)
    
    # convert persian numbers to english
    data['construction_year'] = data['construction_year'].apply(clean_persian_numbers)
    
    # extract year
    data['construction_year'] = data['construction_year'].apply(extract_year)
    
    print(f"  - Valid values: {data['construction_year'].notna().sum()}")
    print(f"  - Year range: {data['construction_year'].min()} to {data['construction_year'].max()}")

Cleaning 'floor' column...
  - Valid values: 541748
  - Unique values: [ 3.  4.  2.  1.  5.  0. 30.  6.  7.  8.]

Cleaning 'rooms_count' column...
  - Valid values: 845899
  - Unique values: [1.0, 2.0, 3.0, 4.0, 5.0]

Cleaning 'construction_year' column...
  - Valid values: 815828
  - Year range: 1370.0 to 1403.0


In [38]:
# create poetry_age column
if 'construction_year' in data.columns:
    current_year = 1404
    
    # calculate property age
    data['property_age'] = current_year - data['construction_year']
    
    # set negative ages to zero
    data['property_age'] = data['property_age'].apply(lambda x: max(0, x) if pd.notna(x) else x)
    
    print(f"\nCreated 'property_age' column")
    print(f"  - Min age: {data['property_age'].min()}")
    print(f"  - Max age: {data['property_age'].max()}")


Created 'property_age' column
  - Min age: 1.0
  - Max age: 34.0


In [39]:
# Identify numeric columns by dtype
numeric_columns_by_dtype = data.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nNumeric columns by dtype ({len(numeric_columns_by_dtype)}):")
for col in numeric_columns_by_dtype:
    print(f"  - {col}")

# Identify object columns
object_columns = data.select_dtypes(include=['object']).columns.tolist()

print(f"\nObject columns ({len(object_columns)}):")
print(object_columns)

# Combine numeric columns identified by dtype and newly created numeric columns
additional_numeric_columns = ['floor','rooms_count', 'construction_year', 'property_age']
additional_numeric_columns = [col for col in additional_numeric_columns if col in data.columns]
all_numeric_columns = list(set(numeric_columns_by_dtype + additional_numeric_columns))
all_numeric_columns.sort()

print(f"\nTotal numeric columns ({len(all_numeric_columns)}):")
for i, col in enumerate(all_numeric_columns, 1):
    print(f"{i:2d}. {col}")



Numeric columns by dtype (22):
  - Unnamed: 0
  - rent_value
  - price_value
  - credit_value
  - transformable_credit
  - transformed_credit
  - transformable_rent
  - transformed_rent
  - land_size
  - building_size
  - floor
  - rooms_count
  - construction_year
  - regular_person_capacity
  - cost_per_extra_person
  - rent_price_on_regular_days
  - rent_price_on_special_days
  - rent_price_at_weekends
  - location_latitude
  - location_longitude
  - location_radius
  - property_age

Object columns (40):
['cat2_slug', 'cat3_slug', 'city_slug', 'neighborhood_slug', 'created_at_month', 'user_type', 'description', 'title', 'rent_mode', 'rent_to_single', 'rent_type', 'price_mode', 'credit_mode', 'rent_credit_transform', 'transformable_price', 'deed_type', 'has_business_deed', 'total_floors_count', 'unit_per_floor', 'has_balcony', 'has_elevator', 'has_warehouse', 'has_parking', 'is_rebuilt', 'has_water', 'has_warm_water_provider', 'has_electricity', 'has_gas', 'has_heating_system', 'has

In [40]:
print("\n" + "="*50)
print("Cleaning Results Summary")
print("="*50)

# missing values
for col in all_numeric_columns:
    if col in data.columns:
        missing = data[col].isnull().sum()
        pct = (missing / len(data)) * 100
        print(f"{col:25s}: {missing:8,d} missing ({pct:.1f}%)")

# display sample of cleaned numeric columns
print("\nSample of cleaned numeric columns:")
sample_cols = ['floor', 'total_floors_count', 'rooms_count', 
               'construction_year', 'property_age']
sample_cols = [col for col in sample_cols if col in data.columns]

print(data[sample_cols].head(10))


Cleaning Results Summary
Unnamed: 0               :        0 missing (0.0%)
building_size            :   19,606 missing (2.0%)
construction_year        :  184,172 missing (18.4%)
cost_per_extra_person    :  989,759 missing (99.0%)
credit_value             :  647,905 missing (64.8%)
floor                    :  458,252 missing (45.8%)
land_size                :  813,604 missing (81.4%)
location_latitude        :  344,392 missing (34.4%)
location_longitude       :  344,392 missing (34.4%)
location_radius          :  660,301 missing (66.0%)
price_value              :  431,654 missing (43.2%)
property_age             :  184,172 missing (18.4%)
regular_person_capacity  :  970,130 missing (97.0%)
rent_price_at_weekends   :  986,449 missing (98.6%)
rent_price_on_regular_days:  981,932 missing (98.2%)
rent_price_on_special_days:  989,537 missing (99.0%)
rent_value               :  648,678 missing (64.9%)
rooms_count              :  154,101 missing (15.4%)
transformable_credit     :  647,915 mi

In [58]:
# cleanData = data[all_numeric_columns]
# cleanData.to_csv('Cleaned_numeric_data.csv', index=False)
data = pd.read_csv('Cleaned_numeric_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 22 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1000000 non-null  int64  
 1   building_size               980394 non-null   float64
 2   construction_year           815828 non-null   float64
 3   cost_per_extra_person       10241 non-null    float64
 4   credit_value                352095 non-null   float64
 5   floor                       541748 non-null   float64
 6   land_size                   186396 non-null   float64
 7   location_latitude           655608 non-null   float64
 8   location_longitude          655608 non-null   float64
 9   location_radius             339699 non-null   float64
 10  price_value                 568346 non-null   float64
 11  property_age                815828 non-null   float64
 12  regular_person_capacity     29870 non-null    float64
 13

In [59]:
# Calculate missing percentage for all columns
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Convert to DataFrame for better visualization
missing_df = pd.DataFrame({
    'column': missing_percentage.index,
    'missing_percentage': missing_percentage.values
}).sort_values('missing_percentage', ascending=False)

# Display columns with high missing values
print("Missing values percentage for all columns:")
print(missing_df.head(20))

Missing values percentage for all columns:
                        column  missing_percentage
3        cost_per_extra_person             98.9759
15  rent_price_on_special_days             98.9537
13      rent_price_at_weekends             98.6449
14  rent_price_on_regular_days             98.1932
12     regular_person_capacity             97.0130
21            transformed_rent             92.7591
20          transformed_credit             92.7591
6                    land_size             81.3604
9              location_radius             66.0301
19          transformable_rent             64.8752
16                  rent_value             64.8678
18        transformable_credit             64.7915
4                 credit_value             64.7905
5                        floor             45.8252
10                 price_value             43.1654
7            location_latitude             34.4392
8           location_longitude             34.4392
11                property_age         

In [60]:
# Identify columns to drop (more than 50% missing)
columns_to_drop = missing_df[missing_df['missing_percentage'] > 50]['column'].tolist()

print(f"\nColumns to drop (>50% missing): {len(columns_to_drop)}")
for col in columns_to_drop:
    print(f"  - {col}: {missing_df[missing_df['column'] == col]['missing_percentage'].values[0]:.1f}% missing")

# Drop these columns
data = data.drop(columns=columns_to_drop)

print(f"\nShape after dropping high-missing columns: {data.shape}")


Columns to drop (>50% missing): 13
  - cost_per_extra_person: 99.0% missing
  - rent_price_on_special_days: 99.0% missing
  - rent_price_at_weekends: 98.6% missing
  - rent_price_on_regular_days: 98.2% missing
  - regular_person_capacity: 97.0% missing
  - transformed_rent: 92.8% missing
  - transformed_credit: 92.8% missing
  - land_size: 81.4% missing
  - location_radius: 66.0% missing
  - transformable_rent: 64.9% missing
  - rent_value: 64.9% missing
  - transformable_credit: 64.8% missing
  - credit_value: 64.8% missing

Shape after dropping high-missing columns: (1000000, 9)


In [62]:
# Get numeric columns (excluding object type columns)
numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nNumeric columns: {len(numeric_columns)}")
print(numeric_columns)

# Calculate missing percentage for numeric columns
numeric_missing = data[numeric_columns].isnull().sum() / len(data) * 100

print("\nMissing values in numeric columns:")
for col, pct in numeric_missing.sort_values(ascending=False).items():
    if pct > 0:
        print(f"  {col}: {pct:.1f}%")


Numeric columns: 9
['Unnamed: 0', 'building_size', 'construction_year', 'floor', 'location_latitude', 'location_longitude', 'price_value', 'property_age', 'rooms_count']

Missing values in numeric columns:
  floor: 45.8%
  price_value: 43.2%
  location_latitude: 34.4%
  location_longitude: 34.4%
  construction_year: 18.4%
  property_age: 18.4%
  rooms_count: 15.4%
  building_size: 2.0%


In [63]:
# First, identify price columns
price_columns = ['price_value', 'transformable_price']
existing_price_cols = [col for col in price_columns if col in data.columns]

print(f"Price columns in dataset: {existing_price_cols}")

# Get numeric columns (excluding price columns for now)
numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns_for_imputation = [col for col in numeric_columns if col not in existing_price_cols]

print(f"\nNumeric columns (excluding price columns): {len(numeric_columns_for_imputation)}")

# Get categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {len(categorical_columns)}")

Price columns in dataset: ['price_value']

Numeric columns (excluding price columns): 8
Categorical columns: 0


In [64]:
# Calculate missing percentage for numeric columns (excluding price columns)
numeric_missing = data[numeric_columns_for_imputation].isnull().sum() / len(data) * 100

# Fill only columns with ≤50% missing
columns_to_fill_numeric = [col for col in numeric_columns_for_imputation 
                          if (numeric_missing[col] > 0) and (numeric_missing[col] <= 50)]

print(f"\nFilling {len(columns_to_fill_numeric)} numeric columns with median:")

for col in columns_to_fill_numeric:
    median_value = data[col].median()
    missing_count_before = data[col].isnull().sum()
    
    data[col] = data[col].fillna(median_value)
    
    print(f"  - {col}: filled {missing_count_before} missing values with median {median_value:.2f}")

# Verify filling
print(f"\nMissing values in numeric columns (excluding price): {data[numeric_columns_for_imputation].isnull().sum().sum()}")


Filling 7 numeric columns with median:
  - building_size: filled 19606 missing values with median 103.00
  - construction_year: filled 184172 missing values with median 1395.00
  - floor: filled 458252 missing values with median 2.00
  - location_latitude: filled 344392 missing values with median 35.72
  - location_longitude: filled 344392 missing values with median 51.35
  - property_age: filled 184172 missing values with median 9.00
  - rooms_count: filled 154101 missing values with median 2.00

Missing values in numeric columns (excluding price): 0


In [65]:
# Check price columns status BEFORE removal
print("\nPrice columns status BEFORE removing rows:")
for price_col in existing_price_cols:
    not_null = data[price_col].notnull().sum()
    null = data[price_col].isnull().sum()
    print(f"  - {price_col}: {not_null} not null, {null} null")

# Count rows before
rows_before = len(data)
print(f"\nRows before removing price-less rows: {rows_before}")

# Remove rows based on price columns
if len(existing_price_cols) == 2:
    # Both price columns exist - remove rows where BOTH are null
    mask = data['price_value'].isnull() & data['transformable_price'].isnull()
    data = data[~mask]
    print("Removed rows where BOTH price_value AND transformable_price are null")
    
elif len(existing_price_cols) == 1:
    # Only one price column exists - remove rows where it's null
    price_col = existing_price_cols[0]
    data = data[data[price_col].notnull()]
    print(f"Removed rows where {price_col} is null")
    
else:
    print("Warning: No price columns found in data!")

# Count rows after
rows_after = len(data)
rows_removed = rows_before - rows_after

print(f"\nRows after removing price-less rows: {rows_after}")
print(f"Rows removed: {rows_removed}")
print(f"Percentage removed: {(rows_removed/rows_before)*100:.1f}%")

# Check price columns status AFTER removal
print("\nPrice columns status AFTER removing rows:")
for price_col in existing_price_cols:
    not_null = data[price_col].notnull().sum()
    null = data[price_col].isnull().sum()
    print(f"  - {price_col}: {not_null} not null, {null} null")


Price columns status BEFORE removing rows:
  - price_value: 568346 not null, 431654 null

Rows before removing price-less rows: 1000000
Removed rows where price_value is null

Rows after removing price-less rows: 568346
Rows removed: 431654
Percentage removed: 43.2%

Price columns status AFTER removing rows:
  - price_value: 568346 not null, 0 null


In [66]:
# Check overall missing values
total_missing = data.isnull().sum().sum()
total_cells = data.shape[0] * data.shape[1]
missing_percent_total = (total_missing / total_cells) * 100

print(f"\nOverall missing values after cleaning:")
print(f"  - Total missing cells: {total_missing}")
print(f"  - Missing percentage: {missing_percent_total:.2f}%")

# Check columns that still have missing values
remaining_missing = data.isnull().sum()
columns_with_missing = remaining_missing[remaining_missing > 0]

if len(columns_with_missing) > 0:
    print(f"\nColumns that still have missing values:")
    for col, count in columns_with_missing.items():
        percent = (count / len(data)) * 100
        print(f"  - {col}: {count} ({percent:.1f}%)")
else:
    print("\nNo missing values remaining in any columns!")


Overall missing values after cleaning:
  - Total missing cells: 0
  - Missing percentage: 0.00%

No missing values remaining in any columns!


In [67]:
print("\n" + "="*60)
print("FINAL DATA CLEANING SUMMARY")
print("="*60)

print(f"1. Initial shape: Unknown (loaded from CSV)")
print(f"2. Final shape: {data.shape}")
print(f"3. Columns dropped (>50% missing): {len(columns_to_drop)}")

# Data types distribution
print(f"\n4. Data types distribution:")
for dtype, count in data.dtypes.value_counts().items():
    print(f"   - {dtype}: {count} columns")

# Price data availability
print(f"\n5. Price data availability:")
if 'price_value' in data.columns:
    print(f"   - price_value: {data['price_value'].notnull().sum()} / {len(data)} rows ({data['price_value'].notnull().sum()/len(data)*100:.1f}%)")
if 'transformable_price' in data.columns:
    print(f"   - transformable_price: {data['transformable_price'].notnull().sum()} / {len(data)} rows ({data['transformable_price'].notnull().sum()/len(data)*100:.1f}%)")

# Sample of final data
print(f"\n6. Sample of final data (first 3 rows):")
print(data.head(3))


FINAL DATA CLEANING SUMMARY
1. Initial shape: Unknown (loaded from CSV)
2. Final shape: (568346, 9)
3. Columns dropped (>50% missing): 13

4. Data types distribution:
   - float64: 8 columns
   - int64: 1 columns

5. Price data availability:
   - price_value: 568346 / 568346 rows (100.0%)

6. Sample of final data (first 3 rows):
   Unnamed: 0  building_size  construction_year  floor  location_latitude  \
1           1           60.0             1384.0    3.0          35.723312   
4           4          115.0             1403.0    4.0          35.723312   
7           7          100.0             1393.0    4.0          35.729832   

   location_longitude   price_value  property_age  rooms_count  
1           51.345791  8.500000e+09          20.0          1.0  
4           51.345791  5.750000e+09           1.0          2.0  
7           51.505466  8.700000e+09          11.0          2.0  


In [68]:
# Save to new CSV file
data.to_csv('final_cleaned.csv', index=False)

In [70]:
# train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data.shape, test_data.shape
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)